In [1]:
from langchain.chains import ConversationalRetrievalChain
from langchain.memory import ConversationBufferMemory
from langchain.vectorstores import Chroma
from langchain.embeddings import HuggingFaceEmbeddings
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, pipeline
from langchain.llms import HuggingFacePipeline
import pickle
import logging

# Setup logging
logging.basicConfig(filename="chatbot_logs.txt", level=logging.INFO)

# FAQs
FAQS = {
    "what is xrp?": "XRP is a cryptocurrency developed by Ripple Labs for real-time global payments.",
    "how to make payment?": "You can make payments using the chatbot interface following these steps..."
}

# 1. Load and validate fragments
def load_fragments(file_path):
    try:
        with open(file_path, "rb") as f:
            fragments = pickle.load(f)
            if not fragments:
                raise ValueError("No fragments found.")
            return fragments
    except FileNotFoundError:
        raise FileNotFoundError(f"Error: Fragments file not found at {file_path}")

fragments = load_fragments("../data/fragments.pkl")

# 2. Create vectorstore
embedding_model = "thenlper/gte-large"
embeddings = HuggingFaceEmbeddings(model_name=embedding_model)
vectorstore = Chroma.from_documents(
    documents=fragments,
    embedding=embeddings,
    persist_directory="./vectorstore"
)

# 3. Load Hugging Face model and pipeline
llm_model_name = "google/flan-t5-large" #"google/flan-t5-base"
tokenizer = AutoTokenizer.from_pretrained(llm_model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(llm_model_name)
hf_pipeline = pipeline("text2text-generation", model=model, tokenizer=tokenizer, max_length=512)
llm = HuggingFacePipeline(pipeline=hf_pipeline)

# 4. Configure memory
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

# 5. Create QA chain
qa_chain = ConversationalRetrievalChain.from_llm(
    llm=llm,
    retriever=vectorstore.as_retriever(),
    memory=memory
)

# 6. Main chatbot function
def chat_with_user():
    print("Chatbot: Hello! How can I assist you today? (Type 'exit' or 'bye' to leave)")
    while True:
        query = input("You: ").strip().lower()
        
        if query in ["exit", "bye"]:
            print("Chatbot: Thank you for chatting with me. Goodbye!")
            break
        
        if query in FAQS:
            response = FAQS[query]
        else:
            try:
                result = qa_chain({"question": query})
                response = result.get("answer", "I'm sorry, I couldn't find an answer.")
            except Exception as e:
                response = f"Chatbot: An error occurred. Please try again later. Error: {e}"
        
        print(f"Chatbot: {response}")
        logging.info(f"User Query: {query}")
        logging.info(f"Chatbot Response: {response}")

# 7. Start chatbot
if __name__ == "__main__":
    chat_with_user()


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Device set to use cpu


Chatbot: Hello! How can I assist you today? (Type 'exit' or 'bye' to leave)


You:  Which course is recommended for beginners?


C:\Users\carol\AppData\Roaming\Python\Python310\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


...

Chatbot: Original Price: $50.00  Discounted Price: $45.00


You:  how to make payment?


Chatbot: You can make payments using the chatbot interface following these steps...


You:  exit


Chatbot: Thank you for chatting with me. Goodbye!
